## General description

In this competition we work on developing solutions for identifying common diseases of cassava (plant). There are 4 different diseases and, of course, a plant could be healhy. As a resutl we have a classification problem with 5 classes.

In this notebook I'll explore the data, train a model using pytorch lightning and analyse the predictions.

![](http://www.naro.go.ug/files/images/crops-naro.jpg)

#### import libraries

In [ ]:
from PIL import Image
from albumentations.core.composition import Compose
from albumentations.pytorch import ToTensorV2
from collections import defaultdict, deque
# from efficientnet_pytorch import EfficientNet
from pytorch_lightning import Callback
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from sklearn import metrics
from sklearn.model_selection import StratifiedKFold
from torch import nn
from torch.autograd import Variable
from torch.utils.data import Dataset, TensorDataset, DataLoader
from torchvision.transforms import functional as F
from typing import Any, Dict, List, Union, Optional
import albumentations as A
import ast
import collections
import copy
import cv2
import datetime
import importlib
import json
import math
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
# import pretrainedmodels
import pytorch_lightning as pl
%matplotlib inline
import random
import seaborn as sns
import shutil
import tempfile
import time
import torch
import torch.distributed as dist
import torch.nn.functional as F
import torch.utils.data
import torchvision
from tqdm.notebook import tqdm
sns.set_style('darkgrid')

#### helper functions

In [ ]:
def visualize(images, transform):
    """
    Plot images and their transformations
    """
    fig = plt.figure(figsize=(32, 16))
    
    for i, im in enumerate(images):
        ax = fig.add_subplot(2, 5, i + 1, xticks=[], yticks=[])
        plt.imshow(im)
        
    for i, im in enumerate(images):
        ax = fig.add_subplot(2, 5, i + 6, xticks=[], yticks=[])
        plt.imshow(transform(image=im)['image'])

def set_seed(seed: int = 42) -> None:
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
set_seed()

## Data Exploration

In [ ]:
path = "../input/cassava-leaf-disease-classification/"

In [ ]:
train = pd.read_csv(f'{path}train.csv')

In [ ]:
with open(f'{path}/label_num_to_disease_map.json', 'r') as f:
    name_mapping = json.load(f)
    
name_mapping = {int(k): v for k, v in name_mapping.items()}
name_mapping

As per description, there are 4 diseases and one class for healthy plants.

In [ ]:
train.head()

In [ ]:
train.shape

In [ ]:
sns.countplot(y=train['label'].map(name_mapping), orient='v')
plt.title('Target distribution');

Interestingly the most common class belongs to one of diseases and not to healthy plants.

### Let's have a look at cassava

At first, lets have a look at images belonging to different classes

In [ ]:
selected_images = []
fig = plt.figure(figsize=(16, 16))
for class_id, class_name in name_mapping.items():
    for i, (idx, row) in enumerate(train.loc[train['label'] == class_id].sample(4).iterrows()):
        ax = fig.add_subplot(5, 4, class_id * 4 + i + 1, xticks=[], yticks=[])
        img = cv2.imread(f"{path}train_images/{row['image_id']}")
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        ax.set_title(f"Image: {row['image_id']}. Label: {row['label']}")
        if i == 0:
            selected_images.append(img)

As far as I can see, one of the common symptoms of desease is a change in color - usually yellow color with different patterns. We will need to be careful with augmentations.

By the way, let's see how different augmentation change the images.

In the first ror where are original images, in the second row there are augmented images. I selected one random image from each class.

In [ ]:
visualize(selected_images, A.HorizontalFlip(p=1))

In [ ]:
visualize(selected_images, A.ShiftScaleRotate(p=1))

In [ ]:
visualize(selected_images, A.Cutout(max_h_size=64, max_w_size=64, p=1))

## Preparing classes for pytorch-lightning

Training neural nets in pytorch-lightning requires writing several classes. Some of them are pure Pytorch classes, some are from pl.

### Dataset class

In [ ]:
class ImageClassificationDataset(Dataset):
    def __init__(
        self,
        image_names: List,
        transforms: Compose,
        labels: Optional[List[int]],
        img_path: str = '',
        mode: str = 'train',
        labels_to_ohe: bool = False,
        n_classes: int = 5,
    ):
        """
        Image classification dataset.

        Args:
            df: dataframe with image id and bboxes
            mode: train/val/test
            img_path: path to images
            transforms: albumentations
        """

        self.mode = mode
        self.transforms = transforms
        self.img_path = img_path
        self.image_names = image_names
        if labels is not None:
            if not labels_to_ohe:
                self.labels = np.array(labels)
            else:
                self.labels = np.zeros((len(labels), n_classes))
                self.labels[np.arange(len(labels)), np.array(labels)] = 1

    def __getitem__(self, idx: int) -> Dict[str, np.array]:
        image_path = self.img_path + self.image_names[idx]
        image = cv2.imread(f'{image_path}', cv2.IMREAD_COLOR)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if image is None:
            raise FileNotFoundError(image_path)
        target = self.labels[idx]

        img = self.transforms(image=image)['image']
        sample = {'image_path': image_path, 'image': img, 'target': np.array(target).astype('int64')}

        return sample

    def __len__(self) -> int:
        return len(self.image_names)

### Augmentations

For now I chose some augmentations at random.

In [ ]:
train_augs = A.Compose([
    A.RandomResizedCrop(height=256, width=256, p=1.0),
    A.Flip(),
    A.RandomBrightnessContrast(),
    A.ShiftScaleRotate(),
    A.Normalize(),
    ToTensorV2(),
])

valid_augs = A.Compose([
    A.Resize(height=256, width=256, p=1.0),
    A.Normalize(),
    ToTensorV2(),
])

### PL datamodule

This class prepares data. Here we initialize data classes and write code for dataloaders. Notice that in `setup` I split data into train and valid.

In [ ]:
class CassavaDataModule(pl.LightningDataModule):
    def __init__(self,
                 df,
                 train_augs,
                 valid_augs,
                 path):
        super().__init__()
        self.df = df
        self.train_augs = train_augs
        self.valid_augs = valid_augs
        self.path = path

    def prepare_data(self):
        pass

    def setup(self, stage=None):
        
        folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
        
        train_indexes, valid_indexes = list(folds.split(self.df, self.df['label']))[0]
        
        train_df = self.df.iloc[train_indexes]
        valid_df = self.df.iloc[valid_indexes]

        
        self.train_dataset = ImageClassificationDataset(image_names=train_df['image_id'].values,
                                                        transforms=train_augs,
                                                        labels=train_df['label'].values,
                                                        img_path=self.path,
                                                        mode='train',
                                                        labels_to_ohe=False,
                                                        n_classes=5)
        self.valid_dataset = ImageClassificationDataset(image_names=valid_df['image_id'].values,
                                                        transforms=valid_augs,
                                                        labels=valid_df['label'].values,
                                                        img_path=self.path,
                                                        mode='valid',
                                                        labels_to_ohe=False,
                                                        n_classes=5)

    def train_dataloader(self):
        train_loader = torch.utils.data.DataLoader(
            self.train_dataset,
            batch_size=64,
            num_workers=4,
            shuffle=True,
        )
        return train_loader

    def val_dataloader(self):
        valid_loader = torch.utils.data.DataLoader(
            self.valid_dataset,
            batch_size=64,
            num_workers=4,
            shuffle=False,
        )

        return valid_loader

    def test_dataloader(self):
        return None

### Defining the model.

I prefer a modular approach for defining the model.

When I train models locally, I don't have to think how to get the weights, but in Kaggle notebooks without internet it is necessary to think about it. I wrote code only for torchvision for now.

In [ ]:
def freeze_until(net: Any, param_name: str = None) -> None:
    """
    Freeze net until param_name


    Args:
        net:
        param_name:

    """
    found_name = False
    for name, params in net.named_parameters():
        if name == param_name:
            found_name = True
        params.requires_grad = found_name

class BasicEncoder(nn.Module):
    def __init__(
        self,
        arch: str = 'resnet18',
        source: str = 'torchvision',
        pretrained: str = None,
        n_layers: int = -2,
        freeze: bool = False,
        to_one_channel: bool = False,
        freeze_until_layer: str = None,
    ) -> None:
        """
        Initialize Encoder.

        Args:
            num_classes: the number of target classes, the size of the last layer's output
            arch: the name of the architecture form pretrainedmodels
            pretrained: the mode for pretrained model from pretrainedmodels
            n_layers: number of layers to keep
            freeze: to freeze model
            freeze_until: freeze until this layer. If None, then freeze all layers
        """
        super().__init__()
        if 'eff' in arch:
            net = EfficientNet.from_pretrained(arch)
            self.output_dimension = net._fc.in_features
        elif source == 'pretrainedmodels':
            net = pretrainedmodels.__dict__[arch](pretrained=pretrained)
            self.output_dimension = list(net.children())[-1].in_features
        elif source == 'torchvision':
            net = torchvision.models.__dict__[arch](pretrained=pretrained)
            net.load_state_dict(torch.load(f'../input/pytorch-pretrained-image-models/{arch}.pth'))
            self.output_dimension = list(net.children())[-1].in_features
            
        if freeze:
            freeze_until(net, freeze_until_layer)

        layers = list(net.children())[:n_layers]
        if to_one_channel:
            # https://www.kaggle.com/c/bengaliai-cv19/discussion/130311#745589
            # saving the weights of the first conv in w
            w = layers[0].weight
            # creating new Conv2d to accept 1 channel
            layers[0] = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
            # substituting weights of newly created Conv2d with w from but we have to take mean
            # to go from  3 channel to 1
            layers[0].weight = nn.Parameter(torch.mean(w, dim=1, keepdim=True))
            layers = nn.Sequential(*layers)

        self.layers = nn.Sequential(*layers)

    def forward(self, x):
        output = self.layers(x)

        return output

In [ ]:
class BasicDecoder(nn.Module):
    def __init__(self, pool_output_size: int = 2, n_classes: int = 1, output_dimension: int = 512) -> None:
        """
        Initialize Decoder.

        Args:
            pool_output_size: the size of the result feature map after adaptive pooling layer
            n_classes: n classes to output
            output_dimension: output dimension of encoder
        """
        super().__init__()
        self.pool = nn.AdaptiveAvgPool2d(output_size=pool_output_size)
        self.fc = nn.Linear(output_dimension * pool_output_size * pool_output_size, n_classes)

    def forward(self, x):
        x = self.pool(x)
        output = self.fc(x.view(x.size()[0], -1))

        return output

In [ ]:
class Net(nn.Module):
    def __init__(self) -> None:
        """
        Model class.

        Args:
            cfg: main config
        """
        super().__init__()
        self.encoder = BasicEncoder(arch='resnet34',
                                    source='torchvision',
                                    pretrained=None,
                                    n_layers=-2,
                                    freeze=False,
                                    to_one_channel=False,
                                    freeze_until_layer=None)
        self.decoder = BasicDecoder(n_classes=5)
        self.loss = nn.CrossEntropyLoss()

    def forward(self, x, targets):
        out = self.encoder(x)
        logits = self.decoder(out)
        loss = self.loss(logits, targets).view(1)
        return logits, loss


### Main pl training class

In this class we define optimizers, schedulers and training itself.

In [ ]:
class LitCassava(pl.LightningModule):
    def __init__(self, model):
        super(LitCassava, self).__init__()
        self.model = model
        self.metric = pl.metrics.Accuracy()
        self.learning_rate = 1e-4

    def forward(self, x, targets, *args, **kwargs):
        return self.model(x, targets)

    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.model.parameters(), lr=self.learning_rate, weight_decay=0.001)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.1, patience=2)

        return (
            [optimizer],
            [{'scheduler': scheduler, 'interval': 'epoch', 'monitor': 'valid_loss'}],
        )

    def training_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'train_loss': loss, f'train_accuracy': score}
        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'train_accuracy': score,
        }

    def training_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)
        
        logs = {'train_loss': avg_loss, 'train_accuracy': score}
        return {'log': logs, 'progress_bar': logs}

    def validation_step(
        self, batch: torch.Tensor, batch_idx: int
    ) -> Union[int, Dict[str, Union[torch.Tensor, Dict[str, torch.Tensor]]]]:
        image = batch['image']
        target = batch['target']
        logits, loss = self(image, target)
        score = self.metric(logits.argmax(1), target)
        logs = {'valid_loss': loss, f'valid_accuracy': score}

        return {
            'loss': loss,
            'log': logs,
            'progress_bar': logs,
            'logits': logits,
            'target': target,
            f'valid_accuracy': score,
        }

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['loss'] for x in outputs]).mean()
        y_true = torch.cat([x['target'] for x in outputs])
        y_pred = torch.cat([x['logits'] for x in outputs])
        score = self.metric(y_pred.argmax(1), y_true)

        # score = torch.tensor(1.0, device=self.device)
        logs = {'valid_loss': avg_loss, f'valid_accuracy': score, 'accuracy': score}
        return {'valid_loss': avg_loss, 'log': logs, 'progress_bar': logs}

## Training the model

In [ ]:
model = Net()

In [ ]:
dm = CassavaDataModule(train, train_augs, valid_augs, f'{path}train_images/')

In [ ]:
trainer = pl.Trainer(
        checkpoint_callback=ModelCheckpoint(monitor='train_loss',
                                            save_top_k=1, filepath='{epoch}_{valid_loss:.4f}_{accuracy:.4f}', mode='min'),
        gpus=1,
        max_epochs=10,
        num_sanity_val_steps=0,
        weights_summary='top',
        callbacks = [EarlyStopping(monitor='valid_loss', patience=10, mode='min')]
)

In [ ]:
lit_model = LitCassava(model)

In [ ]:
trainer.fit(lit_model, dm)

## Analyzing the predictions

First of all, let's make predictions on validation data and collect them.

In [ ]:
lit_model.model.eval();

In [ ]:
image_paths = []
targets = []
predictions = []
for i, batch in tqdm(enumerate(dm.val_dataloader())):
    with torch.no_grad():
        targets.append(batch['target'].detach().cpu().numpy())
        image_paths.append(batch['image_path'])
        pred = lit_model.model(batch['image'], batch['target'])[0]
        predictions.append(pred.detach().cpu().numpy())

In [ ]:
preds_df = pd.DataFrame({'target': np.concatenate(targets),
              'prediction': np.concatenate(predictions).argmax(1),
              'logits': np.concatenate(predictions).max(1),
              'image_paths': [i for j in image_paths for i in j]})
preds_df.head()

In [ ]:
sns.countplot(y=preds_df['prediction'].map(name_mapping), orient='v')
plt.title('Prediction distribution');

In [ ]:
print(metrics.classification_report(preds_df['target'], preds_df['prediction']))

In [ ]:
metrics.confusion_matrix(preds_df['target'], preds_df['prediction'])

In [ ]:
selected_images = []
fig = plt.figure(figsize=(16, 16))
c = 1
for class_id1, class_name1 in name_mapping.items():
    for class_id2, class_name2 in name_mapping.items():
        if class_id1 != class_id2:
            img_path = preds_df.loc[(preds_df['target'] == class_id1)
                                    & (preds_df['prediction'] == class_id2)].sort_values('logits', ascending=False)['image_paths'].values[0]

            ax = fig.add_subplot(5, 4, c, xticks=[], yticks=[])
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            plt.imshow(img)
            ax.set_title(f"Correct class: {class_id1}. Predicted class: {class_id2}")
            c += 1

We can see that the distribution of predictions is similar to the distribution original classes.

Rare classes have more errors in predictions, we will need to find a way to tacke that.

## Prediction

In [ ]:
sub = pd.read_csv(f'{path}/sample_submission.csv')
sub.head()

In [ ]:
test_dataset = ImageClassificationDataset(image_names=sub['image_id'].values,
                                                        transforms=valid_augs,
                                                        labels=sub['label'].values,
                                                        img_path=f'{path}test_images/',
                                                        mode='test',
                                                        labels_to_ohe=False,
                                                        n_classes=5)

test_loader = torch.utils.data.DataLoader(
            test_dataset,
            batch_size=4,
            num_workers=4,
            shuffle=False,
        )



In [ ]:
lit_model.model.cuda()

In [ ]:
predictions = []

for batch in test_loader:

    image = batch['image'].to('cuda')
    target = batch['target'].to('cuda')
    with torch.no_grad():
        outputs = lit_model.model(image, target)[0]
        preds = outputs.argmax(1).detach().cpu().numpy()

        predictions.append(preds)

In [ ]:
sub['label'] = np.concatenate(predictions)

In [ ]:
sub

In [ ]:
sub.to_csv('submission.csv', index=False)